# Librerias

In [1]:
import pandas as pd
import numpy as np
from bokeh.io import curdoc, output_notebook, push_notebook, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, CustomJS
from bokeh.palettes import Spectral6
from bokeh.models.widgets import Slider as BokehSlider  # Rename to avoid conflicts
from bokeh.layouts import column, row
from bokeh.plotting import output_file, show
from bokeh.plotting import figure
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# Datosoutput_file("co2_emissions.html")

In [2]:
url_co2 = 'https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/co2.csv'
co2 = pd.read_csv(url_co2)
url_gm = 'https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/gapminder.csv'
gm = pd.read_csv(url_gm)
df_gm = gm[['Country', 'region']].drop_duplicates()
df_gm.head()
df_w_regions = pd.merge(co2, df_gm, left_on ='country', right_on ='Country', how ='inner')
df_w_regions = pd.merge(co2, df_gm, left_on ='country', right_on ='Country', how ='inner')
df_w_regions = df_w_regions.drop('Country', axis='columns')
new_co2 = pd.melt(df_w_regions, id_vars=['country', 'region'])
columns = ['country', 'region', 'year', 'co2']
new_co2.columns = columns
df_co2 = new_co2[new_co2['year'].astype('int64') > 1963]
df_co2 = df_co2.sort_values(by=['country', 'year'])
df_co2['year'] = df_co2['year'].astype('int64')
df_gdp = gm[['Country', 'Year', 'gdp']]
df_gdp.columns = ['country', 'year', 'gdp']
data = pd.merge(df_co2, df_gdp, on=['country', 'year'], how='left')
data = data.dropna()
data

,country,region,year,co2,gdp
0,Afghanistan,South Asia,1964,0.0863,1182.0
1,Afghanistan,South Asia,1965,0.1010,1182.0
2,Afghanistan,South Asia,1966,0.1080,1168.0
3,Afghanistan,South Asia,1967,0.1240,1173.0
4,Afghanistan,South Asia,1968,0.1160,1187.0
...,...,...,...,...,...
8817,Zimbabwe,Sub-Saharan Africa,2009,0.4060,1352.0
8818,Zimbabwe,Sub-Saharan Africa,2010,0.5520,1484.0
8819,Zimbabwe,Sub-Saharan Africa,2011,0.6650,1626.0
8820,Zimbabwe,Sub-Saharan Africa,2012,0.5300,1750.0


In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
regions_list = data.region.unique().tolist()
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [5]:
source_init = ColumnDataSource(data={
    'x': data.gdp[data.year == min(data.year)],
    'y': data.co2[data.year == min(data.year)],
    'country': data.country[data.year == min(data.year)],
    'region': data.region[data.year == min(data.year)],
    'year': data.year[data.year == min(data.year)] 
})

In [6]:
plot = figure(title='CO2 Emissions vs GDP', y_axis_type='log')


In [7]:
plot.circle(
    x='x', 
    y='y', 
    fill_alpha=0.8, 
    source=source_init, 
    legend_label='region', 
    color=dict(field='region', transform=color_mapper),
    size=7
)

show(plot)

In [8]:
import pandas as pd
import plotly.express as px

fig = px.scatter(data, x='year', y='co2', color='region',
                 labels={
                     'co2': 'CO2 Emissions (Metric Tons per Capita)',
                     'year': 'Year'
                 },
                 title='CO2 Emissions Over Time by Region',
                 marginal_y='box')  


fig


La gráfica muestra las emisiones de CO2 (dióxido de carbono) por persona en diferentes regiones del mundo desde 1970 hasta alrededor de 2010.

No te que América y Europa & Asia Central tienen las emisiones más altas por persona durante la mayor parte del periodo cubierto, con picos que superan ocasionalmente los 40 toneladas métricas por persona. Esto puede reflejar un nivel de industrialización más alto y una mayor dependencia de combustibles fósiles en estas regiones. En el caso de Asia Oriental y Pacífico muestra un incremento significativo en las emisiones por persona a partir del año 2000, lo que podría estar relacionado con el rápido desarrollo económico e industrial de países como China.  Asia del Sur y África Subsahariana presentan las emisiones más bajas por persona, lo que podría indicar una menor industrialización o una infraestructura energética menos dependiente de los combustibles fósiles. Sin embargo, hay que notar el aumento en Asia del Sur después del 2000.por  ultimo, Oriente Medio y Norte de África muestran una variabilidad notable, con algunos años experimentando emisiones muy altas por persona, posiblemente debido a la producción de petróleo y gas en estos lugares.

In [9]:
import pandas as pd
import plotly.express as px


fig = px.scatter(
    data,
    x='gdp', 
    y='co2', 
    animation_frame='year', 
    animation_group='country',
    color='region', 
    hover_name='country',
    labels={'co2': 'CO2 Emissions (Metric Tons per Capita)', 'gdp': 'GDP (US Dollars)'},
    title='Dynamic CO2 Emissions vs GDP by Region',
    marginal_y='box',  
    marginal_x='rug' 

)


fig.update_layout(
    xaxis_title='GDP (US Dollars)',
    yaxis_title='CO2 Emissions (Metric Tons per Capita)',
    xaxis=dict(type='log'), 
    yaxis=dict(range=[0, 100]) 
)


fig


La gráfica muestra la relación entre las emisiones de CO2 per cápita y el PIB por persona en diferentes regiones a lo largo del tiempo. 

   - **Regiones más ricas, más emisiones**: Hay una tendencia general de que las regiones con un PIB más alto por persona tienden a tener también emisiones más altas por persona. Esto puede reflejar el mayor consumo de energía asociado con niveles de vida más altos y economías más industrializadas.
   - **Disparidad en desarrollo y emisiones**: Regiones con PIB más bajo por persona, como África Subsahariana y Asia del Sur, tienden a tener emisiones más bajas por persona, lo que podría indicar una menor industrialización y, por ende, un menor uso de combustibles fósiles.



In [10]:
import plotly.express as px

fig = px.density_contour(
    data,
    x='gdp',
    y='co2',
    color='region',
    labels={'co2': 'CO2 Emissions (Metric Tons per Capita)', 'gdp': 'GDP (US Dollars)'},
    title='Kernel Density Estimation (KDE) Plot',
    marginal_x='rug', 
    marginal_y='histogram',
    hover_name='region'
)

fig.update_layout(
    xaxis_title='GDP (US Dollars)',
    yaxis_title='CO2 Emissions (Metric Tons per Capita)',

)

fig


Concentración baja en PIB alto: La mayoría de las curvas de densidad se concentran hacia el extremo inferior del rango de PIB, indicando que la mayoría de las observaciones de estas regiones tienen un PIB más bajo.
Dispersión y Emisiones: Las curvas de densidad que se extienden hacia mayores valores de PIB (como América y Europa & Asia Central) sugieren una variabilidad mayor tanto en PIB como en emisiones.
Picos y Anchos: La amplitud de las curvas indica la variabilidad del PIB dentro de cada región, mientras que la altura de los picos puede sugerir dónde se concentran más típicamente las observaciones de PIB y emisiones.